#### Dependencies

In [64]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import librosa
import time
import multiprocessing as mp
from dotenv import load_dotenv
from tqdm.notebook import tqdm
import sys
from contextlib import closing
import psutil
import tracemalloc
import threading
import gc

from essentia.standard import (
    MonoLoader,
    Danceability,
    Spectrum,
    FrameCutter,
    Loudness,
    RhythmExtractor2013,
    KeyExtractor,
    Energy,
    TonalExtractor,
    Inharmonicity,
    MFCC,
    OnsetRate,
    SpectralCentroidTime,
    DynamicComplexity,
    SpectralPeaks,
    NoveltyCurve,
    Spectrum,
    FrameGenerator,
    Windowing,
    MelBands,
    BeatsLoudness,
    Beatogram,
    Meter,
)

#### Global Constants

In [65]:
load_dotenv()
DOWNLOAD_FOLDER = os.getenv('DOWNLOAD_FOLDER')
CPU_THREADS = int(os.getenv('CPU_THREADS'))

#### Data

In [66]:
songs_data = pd.read_csv('data/songs_final.csv')

#### Feature Extraction Functions

In [67]:
def create_spectrogram_image(spectrogram_db, sample_rate):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(spectrogram_db, sr=sample_rate, x_axis='time', y_axis='mel', fmax=11025)
    plt.colorbar(format='%+2.0f dB')
    plt.title(f"Mel-Spectrogram")
    plt.tight_layout()
    plt.show()
    plt.close()

In [68]:
def mp3_to_spectrogram(audio_path, sample_rate, create_image=False):
    mp3, _ = librosa.load(audio_path, sr=sample_rate)
    spectrogram = librosa.feature.melspectrogram(y=mp3, sr=sample_rate, n_mels=128, fmax=11025)
    spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)

    if create_image:
        create_spectrogram_image(spectrogram_db, sample_rate)

    return spectrogram_db

In [69]:
def get_mel_bands(audio):
    spectrum = Spectrum()
    frame_generator = FrameGenerator(audio, frameSize=2048, hopSize=1024)
    window = Windowing(type='hann')

    mel_bands = MelBands(numberBands=40)
    mel_band_energies = []

    for frame in frame_generator:
        spec = spectrum(window(frame))
        mel_band_energies.append(mel_bands(spec))

    mel_band_energies = np.array(mel_band_energies)

    return mel_band_energies

In [70]:
def run_essentia_algorithms(audio44k, audio16k):
    _, mfcc_coeffs = MFCC(inputSize=len(audio16k))(audio16k)
    danceability_score = Danceability()(audio44k)
    loudness_score = Loudness()(audio16k)
    bpm, beat_positions, _, _, _ = RhythmExtractor2013(method="multifeature")(audio44k)
    key, scale, _ = KeyExtractor()(audio44k)
    energy_score = Energy()(audio16k)

    ### Chord Significances
    _, _, _, _, chords, _, _, _, _, _, _, _ = TonalExtractor()(audio44k)
    unique_chords, counts = np.unique(chords, return_counts=True)
    chords_significance = {chord: significance for (chord, significance) in zip(unique_chords, counts)}

    ### Inharmonicity
    frames = []
    frameCutter = FrameCutter()
    while True:
        frame = frameCutter(audio44k)
        if not len(frame):
            break
        frames.append(frame)
        
    spectrum_magnitudes = []
    for frame in frames:
        spectrum_magnitudes_frame = Spectrum()(frame)
        spectrum_magnitudes.append(spectrum_magnitudes_frame)
    spectrum_magnitudes = np.array(spectrum_magnitudes).flatten()
    
    frequencies, magnitudes = SpectralPeaks()(audio44k)
    hnr_score = None
    if len(frequencies) > 0 and frequencies[0]: 
        hnr_score = Inharmonicity()(frequencies, magnitudes)
    ###
    
    onset_rate_score = OnsetRate()(audio44k)
    brightness_score = SpectralCentroidTime()(audio44k)
    dynamic_complexity_score, _ = DynamicComplexity()(audio16k)
    
    mel_bands = get_mel_bands(audio44k)
    novelty_curve = NoveltyCurve()(mel_bands)
    novelty_score = np.median(np.abs(np.diff(novelty_curve)))
    
    beats_loudness, beats_loudness_band_ratio = BeatsLoudness(beats=beat_positions)(audio44k)
    beatogram = Beatogram()(beats_loudness, beats_loudness_band_ratio)
    time_signature = Meter()(beatogram)

    features = {
        'Danceability': danceability_score[0],
        'Loudness': loudness_score,
        'BPM': bpm,
        'Key': key,
        'Key Scale': scale,
        'Energy': energy_score,
        'Chords Significance': chords_significance,
        'Inharmonicity': hnr_score,
        'Timbre': np.mean(mfcc_coeffs),
        'Onset Rate': onset_rate_score[1],
        'Brightness': brightness_score,
        'Dynamic Complexity': dynamic_complexity_score,
        'Novelty': novelty_score,
        'Time Signature': time_signature,
    }

    return features

In [71]:
def extract_audio_features(audio_file):
    # Load the audio file
    audio44k = MonoLoader(filename=audio_file)()
    audio16k = MonoLoader(filename=audio_file, sampleRate=16000)()

    # Run algorithms
    algorithm_features = run_essentia_algorithms(audio44k, audio16k)

    # Merge results
    return algorithm_features

#### Util Functions

In [72]:
def get_total_memory_usage(process):
    memory_summary = {f'Process {process.pid}': process.memory_info().rss / (1024 * 1024)}
    for child in process.children(recursive=True):
        memory_summary = memory_summary | {f'Child Process {child.pid}': child.memory_info().rss / (1024 * 1024)}
    return memory_summary

def print_memory_usage(process):
    print(get_total_memory_usage(process))
    snapshot = tracemalloc.take_snapshot()
    print(f"Top Consumer of Process {process.pid}: {snapshot.statistics('lineno')[0]}")

def monitor_memory_usage(process, kill_thread, interval=120):
    while True:
        try:
            if kill_thread.value:
                print("MONITOR THREAD KILLED")
                return
            print_memory_usage(process)
        except Exception as e:
            print(f"Thread ERROR: {e}")
            return
        time.sleep(interval)

#### Main Code

In [73]:
# Class constructed from song path
# Song path must follow this format: /some/path/(int)^(video id)^(title).mp3
#                               e.g  /some/path/0^LlWGt_84jpg^Special Breed.mp3
class SongPath:
    def __init__(self, song_path: str):
        self.path = song_path
        self.filename = os.path.basename(song_path)

        song_filename_split = self.filename.split('^')
        if len(song_filename_split) != 3:
            raise Exception("The song's filename doesn't follow the correct format: /some/path/(int)^(video id)^(title).mp3")
        
        self.index, self.video_id, self.title_with_extension = song_filename_split

        self.index = int(self.index)
        self.title = os.path.splitext(self.title_with_extension)[0]

    def __str__(self):
        return f"Idx: {self.index},  videoID: {self.video_id}, title: {self.title_with_extension}"

In [74]:
def process_song(args):
    song_path, song_results = args
    song = SongPath(song_path)
    song_features = extract_audio_features(song.path)
    song_results.append((song.index, song_features))

In [75]:
def process_songs():                   
    tracemalloc.start()

    song_paths = np.array([os.path.join(DOWNLOAD_FOLDER, song_filename) for song_filename in os.listdir(DOWNLOAD_FOLDER)])

    songs_data_lower, songs_data_higher = [len(song_paths)//6*2, len(song_paths)//6*4]
    song_paths = song_paths[songs_data_lower:songs_data_higher]
    
    with mp.Manager() as manager:
        kill_thread = manager.Value('b', False)
        shared_song_results = manager.list()

        main_process = psutil.Process(os.getpid())
        memory_thread = threading.Thread(target=monitor_memory_usage, args=(main_process, kill_thread))
        memory_thread.start()
        
        with mp.Pool(processes=CPU_THREADS, maxtasksperchild=60) as pool:
            with tqdm(total=len(song_paths), desc="Processing songs") as pbar:
                args = [(song_path, shared_song_results) for song_path in song_paths]
                try:
                    for _ in pool.imap(process_song, args, chunksize=1):
                        pbar.update(1)
                except Exception as e:
                    print(e)
                    
        kill_thread.value = True
        song_results = list(shared_song_results)

    # Aggregate results in the pandas dataframe
    songs_data_full = songs_data.copy(deep=True)
    for song_index, song_features in song_results:
        for feature, value in song_features.items():
            if feature not in songs_data_full.columns and isinstance(value, (tuple, set, list, np.ndarray, dict)):
                songs_data_full[feature] = np.nan
                songs_data_full[feature] = songs_data_full[feature].astype(object)
            songs_data_full.at[song_index, feature] = value

    return songs_data_full

In [76]:
songs_data_full = process_songs()

{'Process 9865': 774.84375, 'Child Process 15788': 729.0703125, 'Child Process 15801': 731.265625}


Processing songs:   0%|          | 0/32782 [00:00<?, ?it/s]

Top Consumer of Process 9865: /tmp/ipykernel_9865/2418371086.py:4: size=55.1 MiB, count=4, average=13.8 MiB
{'Process 9865': 730.01171875, 'Child Process 15788': 730.15234375, 'Child Process 15801': 907.22265625, 'Child Process 15805': 1029.6171875, 'Child Process 15808': 1115.05859375, 'Child Process 15812': 974.5859375}
Top Consumer of Process 9865: /tmp/ipykernel_9865/2418371086.py:4: size=55.1 MiB, count=4, average=13.8 MiB
{'Process 9865': 751.3203125, 'Child Process 15788': 730.4140625, 'Child Process 15801': 991.36328125, 'Child Process 15805': 1145.7578125, 'Child Process 15808': 978.4609375, 'Child Process 15812': 988.30859375}
Top Consumer of Process 9865: /tmp/ipykernel_9865/2418371086.py:4: size=55.1 MiB, count=4, average=13.8 MiB
{'Process 9865': 726.453125, 'Child Process 15788': 730.6015625, 'Child Process 15801': 1003.89453125, 'Child Process 15805': 1000.9921875, 'Child Process 15808': 971.49609375, 'Child Process 15812': 974.16015625}
Top Consumer of Process 9865: /tm

[ WARNING  ] Danceability: Unexpected zero danceability value.


{'Process 9865': 845.27734375, 'Child Process 15788': 972.9921875, 'Child Process 607233': 1419.3515625, 'Child Process 607433': 1152.8671875, 'Child Process 608749': 1162.5625, 'Child Process 609926': 1021.65625}
Top Consumer of Process 9865: /tmp/ipykernel_9865/2418371086.py:4: size=55.1 MiB, count=4, average=13.8 MiB
{'Process 9865': 845.28125, 'Child Process 15788': 973.0234375, 'Child Process 607433': 1311.265625, 'Child Process 608749': 1497.26953125, 'Child Process 609926': 1264.890625, 'Child Process 610246': 1118.234375}
Top Consumer of Process 9865: /tmp/ipykernel_9865/2418371086.py:4: size=55.1 MiB, count=4, average=13.8 MiB
{'Process 9865': 845.28515625, 'Child Process 15788': 973.140625, 'Child Process 608749': 1408.29296875, 'Child Process 609926': 1191.19921875, 'Child Process 610246': 1219.3046875, 'Child Process 610648': 1188.671875}
Top Consumer of Process 9865: /tmp/ipykernel_9865/2418371086.py:4: size=55.1 MiB, count=4, average=13.8 MiB
{'Process 9865': 845.2890625,

In [77]:
songs_data_full.to_csv('data/songs_data_full_4.csv')

In [78]:
songs_data_full.dropna(subset=['Danceability'])


,title,artist,views,videoID,duration,Danceability,Loudness,BPM,Key,Key Scale,Energy,Chords Significance,Inharmonicity,Timbre,Onset Rate,Brightness,Dynamic Complexity,Novelty,Time Signature
3,Justness,Generallykoi,1,J14sCvTWh3Q,86,0.974892,259.176910,99.739212,D,minor,4002.971436,"{'A': 42, 'Ab': 156, 'Abm': 5, 'Am': 41, 'Bbm'...",0.222978,-60.757179,2.142258,654.265686,6.153308,0.122041,4.0
4,INTRANSIGEANCE,BFV,47,uAjBGvZFLi4,228,1.276283,3052.049805,123.974174,C#,minor,158811.843750,"{'A': 214, 'Ab': 44, 'Abm': 553, 'B': 400, 'C#...",0.016665,-31.083698,4.422129,964.104370,5.094929,929.386719,8.0
5,Largehearted,Ratliff Riggs,0,WaLBwUXUEXA,174,1.187087,581.230957,120.000046,C#,minor,13362.679688,"{'A': 1023, 'Ab': 213, 'Abm': 278, 'Am': 4, 'B...",NaN,-23.403543,2.877124,564.821899,3.568006,0.761621,8.0
39,I Cant Sleep at Night,Squeamish,9,TYr3BGqy8HA,160,1.329427,4366.166992,90.379761,C#,minor,271010.062500,"{'A': 149, 'Abm': 71, 'B': 7, 'Bb': 45, 'Bbm':...",0.256316,-12.606949,2.650132,1437.264771,1.242883,5236.390625,2.0
40,The Harmonious Blast Hole Rig,Maritune Art & Music,1,zjJ65T4exxc,126,1.127603,501.769043,114.078415,B,minor,10730.073242,"{'A': 59, 'Abm': 24, 'Am': 7, 'B': 88, 'Bbm': ...",0.017898,-85.319954,1.968254,315.806366,5.031539,0.050219,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69671,Seppuku,ShogunF,432320,Uj3riPB_8aI,177,1.313095,4802.254883,109.988892,G,minor,312387.906250,"{'A': 37, 'Am': 9, 'C': 337, 'Cm': 44, 'D': 63...",NaN,-14.756376,4.905864,949.604980,2.937516,2619.375000,4.0
69672,Made In Thailand,Carabao,179003,I-vHdSa6b_M,223,1.342756,1956.814331,110.109093,D,minor,81801.359375,"{'A': 41, 'Ab': 2, 'Abm': 50, 'Am': 720, 'B': ...",0.006618,-29.508310,4.256273,917.853882,2.453418,139.285889,4.0
69673,Regenbogen (Digimon Tamers),Anime Allstars,171958,iVPYO36xk_Y,259,1.137065,3071.331543,172.265701,A,major,160311.656250,"{'A': 2194, 'Ab': 6, 'Abm': 2, 'Am': 90, 'B': ...",0.021340,-15.630503,3.351934,1204.318726,2.339727,207.432129,8.0
69674,Dream Of You (feat. Peter Heppner),Schiller,421943,_0AVOVzKUPY,241,1.258478,5232.844727,118.215691,C#,minor,355103.406250,"{'A': 144, 'Ab': 436, 'Abm': 968, 'B': 636, 'B...",0.226818,-17.872032,4.292119,1329.429565,3.003580,1830.199219,3.0
